In [ ]:
import cv2
import numpy as np

def resize_image(input_image, target_size):
    """
    Resize the input image to the target size.
    """
    return cv2.resize(input_image, target_size)

def create_jigsaw_puzzle(input_image, grid_size=(3, 3), target_size=None):
    if target_size:
        input_image = resize_image(input_image, target_size)

    # Split the input image into a grid of puzzle pieces.
    puzzle_pieces = []
    puzzle_piece_size = input_image.shape[0] // grid_size[0], input_image.shape[1] // grid_size[1]
    
    # Create a list of indices representing a row-wise shuffle.
    indices = list(range(grid_size[0] * grid_size[1]))
    np.random.shuffle(indices)
    
    for i, idx in enumerate(indices):
        row, col = divmod(i, grid_size[1])
        x_start = col * puzzle_piece_size[1]
        x_end = (col + 1) * puzzle_piece_size[1]
        y_start = row * puzzle_piece_size[0]
        y_end = (row + 1) * puzzle_piece_size[0]
        puzzle_pieces.append(input_image[y_start:y_end, x_start:x_end])

    # Create a solved image of the puzzle.
    solved_image = np.zeros((input_image.shape[0], input_image.shape[1], input_image.shape[2]), dtype=np.uint8)
    for row in range(grid_size[0]):
        for col in range(grid_size[1]):
            solved_image[row * puzzle_piece_size[0]:(row + 1) * puzzle_piece_size[0],
                          col * puzzle_piece_size[1]:(col + 1) * puzzle_piece_size[1]] = puzzle_pieces[row * grid_size[1] + col]

    # Create an unsolved image by shuffling the pieces.
    unsolved_image = np.copy(solved_image)
    np.random.shuffle(puzzle_pieces)
    for row in range(grid_size[0]):
        for col in range(grid_size[1]):
            unsolved_image[row * puzzle_piece_size[0]:(row + 1) * puzzle_piece_size[0],
                            col * puzzle_piece_size[1]:(col + 1) * puzzle_piece_size[1]] = puzzle_pieces[row * grid_size[1] + col]

    return puzzle_pieces, solved_image, unsolved_image

# Example usage:
input_image = cv2.imread("dog.jpg")

# Resize the input image to a specific target size, e.g., (800, 600).
target_size = (800, 600)
puzzle_pieces, solved_image, unsolved_image = create_jigsaw_puzzle(input_image, grid_size=(3, 3), target_size=target_size)

# Save the puzzle pieces, solved image, and unsolved image to disk.
for i in range(len(puzzle_pieces)):
    cv2.imwrite(f"puzzle_piece_{i}.jpg", puzzle_pieces[i])
cv2.imwrite("solved_image.jpg", solved_image)
cv2.imwrite("unsolved_image.jpg", unsolved_image)

# Display the unsolved image and solved image.
cv2.imshow("Unsolved Image", unsolved_image)
cv2.imshow("Solved Image", solved_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
